## Challenge: Exercise 4
#### To match searches with bookings

For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import datetime as dt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import bz2
files_path = "/home/dsc/Repositories/Master-in-Data-Science/Data/"

##### Loading and formatting bookings

In [3]:
# Defining new searches file
new_searches_file = files_path+"new_searches.csv"
new_searches_df = pd.DataFrame([])

searches_file = bz2.BZ2File(files_path+"searches.csv.bz2")

n=0
start = dt.datetime.now()

# Loading searches file using chunks due to limit of memory 
for searches_df in pd.read_csv(searches_file, sep='^', chunksize=1e6,iterator=True):
    searches_df['Seg1Date'] = pd.to_datetime(searches_df['Seg1Date'])
    searches_df['Is_Booked'] = 0

    # Reseting index avoids errors deleting duplicates after merging dataframes
    searches_df.reset_index(inplace=True)
    
    bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")

    # Loading bookings file using chunks due to limit of memory
    for booking_df in pd.read_csv(bookings_file,sep='^',chunksize=1e6,iterator=True,
                                  usecols=['dep_port','arr_port','brd_time           ','pax']):
        
        # Discarding blanks, negatives bookings anf garbage rows
        booking_df.dropna(inplace=True)
        booking_df = booking_df[booking_df['pax'] >=1]
        booking_df.drop('pax', axis=1, inplace=True)
        booking_df = booking_df[booking_df['brd_time           '] != '3']

        # Formatting columns of bookings dataframe to match columns of searches dataframe
        booking_df.columns={'Origin','Destination','Seg1Date'}
        booking_df['Origin'] = booking_df['Origin'].str.strip()
        booking_df['Destination'] = booking_df['Destination'].str.strip()
        booking_df['Seg1Date'] = pd.to_datetime(booking_df['Seg1Date'])
        booking_df['Seg1Date'] = booking_df['Seg1Date'].dt.date
        booking_df['Seg1Date'] = pd.to_datetime(booking_df['Seg1Date'])

        # Adding requested column "if search = booking"
        booking_df['Is_Booked_Chunk'] = 1
        
        # Obtaining new searches with bookings
        searches_df = pd.merge(searches_df,booking_df,on=['Origin','Destination','Seg1Date'],how="left").drop_duplicates()
        searches_df['Is_Booked_Chunk'].fillna(0,inplace=True)
        # Adding only new matches bookings vs searches
        searches_df['Is_Booked'] = searches_df['Is_Booked_Chunk'].where(searches_df['Is_Booked_Chunk']==1,other=searches_df['Is_Booked'])
        searches_df.drop('Is_Booked_Chunk', axis=1, inplace=True)
        
        print 'booking {} seconds: completed'.format((dt.datetime.now() - start).seconds)

    # Writing new searches csv file
    searches_df.drop('index', axis=1, inplace=True)
    if n == 0:
        searches_df.to_csv(new_searches_file,sep='^',mode='w')
    else:
        searches_df.to_csv(new_searches_file,sep='^',mode='a',header=False)
    
    print '{} search {} seconds: completed'.format(n,(dt.datetime.now() - start).seconds)
    n += 1
    

booking 44 seconds: completed
booking 73 seconds: completed
booking 101 seconds: completed
booking 130 seconds: completed
booking 158 seconds: completed
booking 187 seconds: completed
booking 222 seconds: completed
booking 267 seconds: completed
booking 308 seconds: completed
booking 351 seconds: completed
booking 358 seconds: completed
0 search 374 seconds: completed

/usr/lib64/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/usr/lib64/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



booking 428 seconds: completed
booking 457 seconds: completed
booking 485 seconds: completed
booking 514 seconds: completed
booking 543 seconds: completed
booking 571 seconds: completed
booking 601 seconds: completed
booking 630 seconds: completed
booking 659 seconds: completed
booking 687 seconds: completed
booking 694 seconds: completed
1 search 712 seconds: completed
booking 759 seconds: completed
booking 788 seconds: completed
booking 817 seconds: completed
booking 847 seconds: completed
booking 876 seconds: completed
booking 905 seconds: completed
booking 935 seconds: completed
booking 965 seconds: completed
booking 994 seconds: completed
booking 1023 seconds: completed
booking 1030 seconds: completed
2 search 1046 seconds: completed
booking 1091 seconds: completed
booking 1121 seconds: completed
booking 1150 seconds: completed
booking 1180 seconds: completed
booking 1210 seconds: completed
booking 1240 seconds: completed
booking 1270 seconds: completed
booking 1299 seconds: comp

/usr/lib64/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
!bzip2 {files_path+"new_searches.csv"}